# Run whole image QC pipeline in CellProfiler

To determine if there are images that of poor quality, we run a CellProfiler pipeline specific to extracting image quality metrics.
We extract blur and saturation metrics, we can use to identify thresholds for these metrics to separate the good and poor quality images.


## Import libraries

In [2]:
import pathlib
import pprint
import sys

# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd
else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break
sys.path.append(str(root_dir / "utils"))
from notebook_init_utils import avoid_path_crash_bandicoot, init_notebook

root_dir, in_notebook = init_notebook()
sys.path.append(f"{root_dir}/utils")
import cp_parallel

## Set paths and variables

In [ ]:
bandicoot_path = pathlib.Path("~/mnt/bandicoot").resolve()
if bandicoot_path.exists():
    # comment out depending on whose computer you are on
    # mike's computer
    image_base_dir = pathlib.Path("~/mnt/bandicoot").resolve()
else:
    image_base_dir = root_dir

In [6]:
# set the run type for the parallelization
run_name = "quality_control"

# set path for pipeline for whole image QC
path_to_pipeline = pathlib.Path(
    f"{root_dir}/1.image_quality_control/pipeline/whole_image_qc.cppipe"
).resolve(strict=True)

# set main output dir for all plates if it doesn't exist
output_dir = pathlib.Path(f"{root_dir}/1.image_quality_control/pipeline/qc_results")
output_dir.mkdir(exist_ok=True)

# directory where images are located within folders (parent folder is the plate and the child folders are wells containing images)
images_dir = pathlib.Path(f"{image_base_dir}/data").resolve(strict=True)

# filter plate names to include only those starting with 'NF' or 'SARCO'
plate_names = []
for file_path in images_dir.iterdir():
    plate_name = str(file_path.stem)

    if plate_name.startswith("NF") or plate_name.startswith("SARCO"):
        print("Plate name:", plate_name)
        plate_names.append(plate_name)

print("There are a total of", len(plate_names), "plates. The names of the plates are:")
for plate in plate_names:
    print(plate)

Plate name: NF0030_T1
Plate name: NF0016_T1
Plate name: SARCO219_T2
Plate name: NF0014_T1
Plate name: NF0018_T6
Plate name: NF0021_T1
Plate name: NF0040_T1
Plate name: SARCO361_T1
There are a total of 8 plates. The names of the plates are:
NF0030_T1
NF0016_T1
SARCO219_T2
NF0014_T1
NF0018_T6
NF0021_T1
NF0040_T1
SARCO361_T1


## Generate dictionary with plate info to run CellProfiler

In [3]:
# filter plate names to include only those that haven't been processed (output directory doesn't exist or is empty)
unprocessed_plate_names = [
    name
    for name in plate_names
    if not (output_dir / f"{name}_qc_results").exists()
    or not any((output_dir / f"{name}_qc_results").iterdir())
]

# create plate info dictionary with all parts of the CellProfiler CLI command
plate_info_dictionary = {
    name: {
        "path_to_images": pathlib.Path(
            list(images_dir.rglob(f"{name}/raw_images"))[0]
        ).resolve(strict=True),
        "path_to_output": pathlib.Path(f"{output_dir}/{name}_qc_results"),
        "path_to_pipeline": path_to_pipeline,
    }
    for name in unprocessed_plate_names
}

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'SARCO219': {   'path_to_images': PosixPath('/media/18tbdrive/1.Github_Repositories/GFF_3D_organoid_profiling_pipeline/data/SARCO219/raw_images'),
                    'path_to_output': PosixPath('../qc_results/SARCO219_qc_results'),
                    'path_to_pipeline': PosixPath('/media/18tbdrive/1.Github_Repositories/GFF_3D_organoid_profiling_pipeline/1.image_quality_control/pipeline/whole_image_qc.cppipe')},
    'SARCO361': {   'path_to_images': PosixPath('/media/18tbdrive/1.Github_Repositories/GFF_3D_organoid_profiling_pipeline/data/SARCO361/raw_images'),
                    'path_to_output': PosixPath('../qc_results/SARCO361_qc_results'),
                    'path_to_pipeline': PosixPath('/media/18tbdrive/1.Github_Repositories/GFF_3D_organoid_profiling_pipeline/1.image_quality_control/pipeline/whole_image_qc.cppipe')}}


## Run QC pipeline in CellProfiler

In [ ]:
cp_parallel.run_cellprofiler_parallel(
    plate_info_dictionary=plate_info_dictionary, run_name=run_name
)